In [2]:
import pandas as pd
from evaluate import load
rouge = load('rouge')
bertscore = load("bertscore")

2025-05-23 08:05:48.027001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747973148.041911 2874480 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747973148.046460 2874480 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747973148.058033 2874480 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747973148.058047 2874480 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747973148.058049 2874480 computation_placer.cc:177] computation placer alr

In [34]:
method = 'sft'
model_list = [
    # 'Qwen2.5-7B-Instruct', 
    # 'Llama-3.1-8B-Instruct',
    # 'gemma-2-9b-it', 
    # 'llama3-8b-cpt-sahabatai-v1-instruct', 
    'Llama-SEA-LION-v3-8B-IT'
]
train_set = 'jvsu_llm_filtered'
language = 'jv'

rouge_scores = []
bertscores = []
for model in model_list:
    if method == 'sft':
        df = pd.read_csv(f'src/generator/predictions/{method}/{model}_{train_set}.csv')
        if language == 'jv':
            df = df.iloc[:1094]
        else:
            df = df.iloc[1904:]
    else:
        df = pd.read_csv(f'src/generator/predictions/{method}/{model}_{language}.csv')
    correct_endings = df['correct_ending'].to_list()
    predicted_endings = df['predicted'].to_list()

    rouge_score = rouge.compute(predictions=predicted_endings, references=correct_endings)

    results = bertscore.compute(predictions=predicted_endings, references=correct_endings, model_type='bert-base-multilingual-cased')
    def get_mean(l):
        return sum(l) / len(l)
    bert_score = {
        "precision": get_mean(results['precision']),
        "recall": get_mean(results['recall']),
        "f1": get_mean(results['f1'])
    }
    rouge_scores.append(rouge_score['rougeL'])
    bertscores.append(bert_score['f1'])

In [35]:
for bs, rl in zip(bertscores, rouge_scores):
    print(f"{bs*100:.2f} / {rl*100:.2f}")

71.23 / 18.63
